In [ ]:
import json
import numpy as np 
from matplotlib import pyplot as plt

In [1]:
 
file_head = '../Baselines/EnemyPositionPrediction/checkpoints'

## 可更换文件位置 /ELPNet/full/all/loss.json, /ELPNet/trans/all/loss.json, /ELPNet/gnn/all/loss.json, /ELPNet/base/all/loss.json
ELP_loss = '/ELPNet/full/all/loss.json'    

# ELPNet 模型
with open(file_head + ELP_loss,'r') as f:
    ELP_loss_file = json.load(f)




FileNotFoundError: [Errno 2] No such file or directory: '../Baselines/EnemyPositionPrediction/checkpoints/CNN_GRU/all/loss.json'

In [4]:
def smooth(data_np, n_point=1000):
#     weight=np.ones(n_point)/n_point
    last = np.ones(n_point-1)*data_np[0]
    data_np = np.concatenate((last,data_np),axis=0)
    smoothed = []
    for index in range(len(data_np)-n_point+1):
        
        smoothed_val = data_np[index:index+n_point].mean()
        smoothed.append(smoothed_val)
    return np.array(smoothed)


In [5]:
# ELPNet 模型
ELP_record_loss_mean, \
ELP_record_loss, \
ELP_record_loss_step, \
ELP_record_acc,\
ELP_record_acc_step = smooth(np.array(ELP_loss_file['record_loss'])), \
                            np.array(ELP_loss_file['record_loss']),\
                            np.arange(len(ELP_loss_file['record_loss_step'])),\
                            np.array(ELP_loss_file['record_acc']), \
                            np.arange(len(ELP_loss_file['record_acc_step']))



print(len(ELP_record_loss_step))

# loss_index1 = np.arange(2*len(ELP_record_loss_step))
loss_index = np.arange(len(ELP_record_loss_step))

#######  添加
print(len(ELP_record_acc_step))
acc_index = np.arange(len(ELP_record_acc_step))


52515
13018


In [4]:

fig = plt.figure()
# plt.title("Mean loss of training") 
plt.xlabel("step") 
plt.ylabel("cross entropy") 


plt.plot(loss_index, ELP_record_loss[loss_index],linewidth=0.5) 
plt.plot(loss_index, ELP_record_loss_mean[loss_index], label='ELP-Net', color = 'b',linewidth=1) 


plt.legend()
plt.axis([-1000, 40000, -0.5, 9])
plt.show()
fig.savefig('loss.png',format='png')
fig.savefig('loss.eps',format='eps')

NameError: name 'plt' is not defined

In [3]:

fig = plt.figure()
# plt.title("Mean accuracy of training") 
plt.xlabel("episodes") 
plt.ylabel("accuracy") 

plt.plot(acc_index,smooth(ELP_record_acc[acc_index], 100), label='ELP-Net') 

plt.axis([0, 8000, 0, 1])
plt.legend(loc='upper center', bbox_to_anchor=(0.5,1),ncol=1,fancybox=True,shadow=False)
# plt.legend()
plt.show()
fig.savefig('acc.png',format='png')
fig.savefig('acc.eps',format='eps')

NameError: name 'plt' is not defined

In [ ]:
import pickle
import numpy as np 
from matplotlib import pyplot as plt 

In [49]:

file_head = '../Baselines/EnemyPositionPrediction/checkpoints'
ELP_result = '/ELPNet/full/all/test/test_result'


with open(file_head + ELP_result,'rb') as f:
    ELP_result_file = pickle.load(f)


ELP_path = '/ELPNet/full/all/test/path_list'

with open(file_head + ELP_path,'rb') as f:
    ELP_path_file = pickle.load(f)


In [50]:
def get_max_acc(result_file, path_file, offset=0):
    acc_n = []
    for dic in result_file:
        acc = [[] for _ in range(7)]
        for piece in range(6):
            action_pre=dic['action_pre_per_replay'][piece]
            action_gt = dic['action_gt_per_replay'][piece]
            for action_pre_per_replay, action_gt_per_replay in zip(action_pre, action_gt):
                acc_per_replay = np.mean(action_pre_per_replay[offset:] == action_gt_per_replay[offset:])
                acc[piece].append(acc_per_replay)

        for piece in range(6):
            acc[piece] = np.array(acc[piece]).mean()
        acc[6] = np.array(acc[:6]).mean()
        acc_n.append(acc)
    acc_n = np.array(acc_n)
    
    for i in range(acc_n.shape[0]):
        print(acc_n[i], path_file[i])  
    max_row = np.argmax(acc_n[:,-1])
    print(path_file[max_row])
    ##################################
    steps = 10
    dic_max = result_file[max_row]
    
    acc_n_step = []
    action_pre_result, action_gt_result = [[] for _ in range(steps)], [[] for _ in range(steps)]
    for piece in range(6):
        action_pre = dic_max['action_pre_per_replay'][piece]
        action_gt = dic_max['action_gt_per_replay'][piece]
        for action_pre_per_replay, action_gt_per_replay in zip(action_pre, action_gt):
            if len(action_pre_per_replay[offset:]) < steps:
                continue

            step = len(action_pre_per_replay[offset:]) // steps
            for s in range(steps):
                action_pre_result[s].append(action_pre_per_replay[offset:][s * step:(s + 1) * step])
                action_gt_result[s].append(action_gt_per_replay[offset:][s * step:(s + 1) * step])

        X = np.arange(steps)
        Y = np.zeros(steps)
        for idx, (action_pres, action_gts) in enumerate(zip(action_pre_result, action_gt_result)):

            action_pres_np = np.hstack(action_pres)
            action_gts_np = np.hstack(action_gts)

            Y[idx] = np.mean(action_pres_np == action_gts_np)
        acc_n_step.append(Y)
    return acc_n, acc_n[max_row, :], np.array(acc_n_step)

In [51]:
ELP_acc, ELP_acc_max, ELP_acc_n_step = get_max_acc(ELP_result_file, ELP_path_file, offset=0)


print(ELP_acc_max)

[0.27906341 0.24628919 0.24858493 0.28625769 0.25242821 0.29660851
 0.26820532] checkpoints\CNN_GRU\all\snapshots\model_iter_1680480.pth
[0.28176386 0.24212984 0.25883947 0.28032756 0.26300598 0.29384742
 0.26998569] checkpoints\CNN_GRU\all\snapshots\model_iter_1650528.pth
[0.27405225 0.24619773 0.24974883 0.27376781 0.25251396 0.28819829
 0.26407981] checkpoints\CNN_GRU\all\snapshots\model_iter_1600512.pth
[0.28155891 0.24698434 0.25462867 0.27983122 0.26205304 0.29325191
 0.26971801] checkpoints\CNN_GRU\all\snapshots\model_iter_1550496.pth
[0.28075498 0.24265648 0.25018639 0.27996168 0.25665222 0.29870508
 0.2681528 ] checkpoints\CNN_GRU\all\snapshots\model_iter_1500480.pth
[0.28135178 0.24750152 0.25407168 0.27887831 0.26196392 0.29254749
 0.26938578] checkpoints\CNN_GRU\all\snapshots\model_iter_1450464.pth
[0.28371037 0.24493913 0.26353223 0.29230815 0.26936652 0.30648025
 0.27672277] checkpoints\CNN_GRU\all\snapshots\model_iter_1400448.pth
[0.2766249  0.2495049  0.25936265 0.28307

In [2]:

fig = plt.figure()
# plt.title("Mean accuracy of training") 
plt.xlabel("network parameters with time") 
plt.ylabel("accuracy") 

plt.plot(np.arange(len(ELP_acc)),ELP_acc[:,-1], '-^', label='ELPNet') 

plt.axis([0, 40, 0.15, 0.8])
plt.legend(loc='upper center', bbox_to_anchor=(0.8,1),ncol=1,fancybox=True,shadow=False)
# plt.legend()
plt.show()
fig.savefig('acc_parameter.png',format='png')
fig.savefig('acc_parameter.eps',format='eps')

NameError: name 'plt' is not defined

In [1]:
N = 7
fig = plt.figure()
index = np.arange(0, N, 1)
print(ELP_acc_max)
tick_label = ['tank_1','tank_2','armored\nvehicle_1','armored\nvehicle_2','soldier_1','soldier_2','mean']; 
bar_width = 0.3

plt.bar(index , ELP_acc_max, width=bar_width,label='ELP-Net', tick_label = tick_label )
for a, b in zip(index , ELP_acc_max):
    plt.text(a, b + 0.01, '%.3f' % b, ha='center', va='bottom', fontsize=10)


plt.legend(loc='upper center') #loc='upper center', bbox_to_anchor=(0.8,0.5),ncol=1,fancybox=True,shadow=False)
plt.xlim(-0.5,7);
plt.ylim(0,0.7);
plt.tick_params(labelsize=7)
plt.xlabel("pieces") 
plt.ylabel("accuracy") 
plt.show()
fig.savefig('acc_piece.png',format='png')
fig.savefig('acc_piece.eps',format='eps')



NameError: name 'plt' is not defined

In [5]:

fig = plt.figure()
title_label = ['Tanks_1','Tanks_2','Armored vehicles_1','Armored vehicles_2','Soldiers_1','Soldiers_2']; 


acc = np.mean(ELP_acc_n_step, axis=0)

   

plt.xlabel("stages") 
plt.ylabel("accuracy") 
x_index = np.arange(1,11)

plt.plot(x_index, acc, '*-', label='ELP-Net') 

plt.legend()
plt.axis([0, 11, 0, 0.8])
plt.xticks(x_index)
plt.show()
fig.savefig('acc_step.png',format='png')
fig.savefig('acc_step.eps',format='eps')

NameError: name 'plt' is not defined

In [55]:
file_head = '../Baselines/EnemyPositionPrediction/checkpoints'

ELP_result = '/ELPNet/full/all/test_pro/test_result'


with open(file_head + ELP_result,'rb') as f:
    ELP_result_file = pickle.load(f)



In [56]:
def show_test_pro_result(result, steps=10, offset=0):
    action_pres, action_gts = result
    # print(action_pres)
    # print(action_gts)

    ################################## Calc Acc #########################################
    
    action_pres_np = np.array(action_pres, dtype=object)
    
    # print(action_pres_np)
    action_gts_np = np.array(action_gts, dtype=object)
    
    # print(action_pres_np.shape, action_gts_np.shape)
    acc_type = [1,3,5,10,15,50]
    action_acc = []
    for piece in range(6):
        acc_piece = []
        for replay in range(action_pres_np.shape[1]):
            acc_replay = []
            action_pres = action_pres_np[piece, replay]
            action_gts = action_gts_np[piece, replay]
            for i, num in enumerate(acc_type):
                
                acc_replay.append(np.mean([pos in action_pres[j, -num:]
                                           for j, pos in enumerate(action_gts[offset:])]))
            acc_piece.append(acc_replay)
    
        action_acc.append(acc_piece)
    action_acc = np.array(action_acc)
    print(action_acc.shape)
    action_acc = action_acc.mean(axis=1)
    print('\tAction Accuracy: {}\t'.format(action_acc))


#     print(Y, Y[:,0].shape, X.shape)
    
    return action_acc

In [57]:
ELP_pre = ELP_result_file['action_pre_per_replay']
ELP_gt = ELP_result_file['action_gt_per_replay']
ELP_acc = show_test_pro_result((ELP_pre, ELP_gt), offset=0)

(6, 220, 6)
	Action Accuracy: [[0.29818145 0.42728855 0.50075724 0.61419573 0.69557932 0.86420383]
 [0.26873069 0.40273118 0.47728406 0.60592602 0.68803942 0.88121118]
 [0.28821319 0.38587742 0.45958385 0.57804725 0.66334956 0.8603927 ]
 [0.30266645 0.4316171  0.50411864 0.61897592 0.69699349 0.89640833]
 [0.2998503  0.39664106 0.4654846  0.58527766 0.67111039 0.86409127]
 [0.31760039 0.44149625 0.51925503 0.63572589 0.72398123 0.90570924]]	


In [6]:
N = 6
index = np.arange(0, N, 1)
label = ['tank_1','tank_2','armored\nvehicle_1','armored\nvehicle_2','soldier_1','soldier_2']


# 创建一个3行2列的子图网格
fig, axs = plt.subplots(3, 2, figsize=(10, 7))  # 可以根据需要调整figsize

# 绘制每个子图
for i in range(6):
    row = (i // 2)  # 计算行位置
    col = (i % 2)  # 计算列位置
    ax = axs[row, col]
    
    tick_label = ['top 1','top 3','top 5','top 10','top 15','top 50']
    bar_width = 0.3
    
    ax.bar(index, ELP_acc[i], width=bar_width, label='ELP-Net', tick_label=tick_label)
    for a, b in zip(index, ELP_acc[i]):
        ax.text(a, b + 0.01, '%.3f' % b, ha='center', va='bottom', fontsize=8)
    
    ax.set_xlim(-0.5, N-0.5)
    ax.set_ylim(0, 1)
    ax.tick_params(labelsize=9)
    ax.set_xlabel("test dataset accuracy")
    ax.set_ylabel(label[i], rotation='horizontal', verticalalignment='center', horizontalalignment='right')

# 调整子图间的间距
plt.tight_layout()

# 显示图例
# 由于图例可能重叠，我们可以选择只在最后一个子图上显示图例
axs[2, 1].legend(bbox_to_anchor=(1.05, 4), loc='upper center', fontsize=10)

# 显示图形
plt.show()

# 保存图形
fig.savefig('acc_compare.png', format='png')
fig.savefig('acc_compare.eps', format='eps')

NameError: name 'np' is not defined

In [7]:
ELP_acc_top_means = np.mean(ELP_acc, axis=0)

print(ELP_acc_top_means)
N = 6
fig = plt.figure()
index = np.arange(0, N, 1)

tick_label = ['top 1','top 3','top 5','top 10','top 15','top 50']; 
bar_width = 0.15
plt.bar(index , ELP_acc_top_means, width=bar_width,label='ELP-Net', tick_label = tick_label)


for a, b in zip(index  , ELP_acc_top_means):
    plt.text(a, b + 0.01, '%.3f' % b, ha='center', va='bottom', fontsize=10)


plt.legend(loc='upper left') #loc='upper center', bbox_to_anchor=(0.8,0.5),ncol=1,fancybox=True,shadow=False)
plt.xlim(-0.5,6)
plt.ylim(0,1)
plt.tick_params(labelsize=7)
plt.xlabel("test top mean accuracy") 
plt.ylabel("accuracy") 
plt.show()
fig.savefig('ELP_acc_top_means.png',format='png')
fig.savefig('ELP_acc_top_means.eps',format='eps')


NameError: name 'np' is not defined